### Live Inference for Anomaly Detection for Machine Operating Sounds (ADMOS)

In [ ]:
# importing libraries

# basic libraries
import numpy as np
import scipy
from scipy.signal import spectrogram
import wave
import matplotlib
import matplotlib.pyplot as plt

# custom functions
import common_pynq as com

# threading
from threading import Timer
from time import sleep, perf_counter

# for profiling
import cProfile
import pstats
import io
import os

In [ ]:
# importing bitfile

from pynq import Overlay
overlay = Overlay('design_1_wrapper.bit')
overlay.audio = overlay.audio_codec_ctrl_0

In [ ]:
# configuring audio codec
# note: takes a while, patience :)
# dont run other lines until this is done

overlay.audio.configure()

In [ ]:
# selecting input and checking sample rate
# note: codec is fixed at 48kHz however nn is trained with 16kHz
# but this is not an issue as we will use 16kHz timer to update buffer

print("selecting input")
overlay.audio.select_line_in()
sample_rate = overlay.audio.sample_rate
print("sample rate: ")
print(sample_rate)

In [ ]:
# initializing timer object from threading

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.profile = kwargs.get('rt_profile', False)
        if self.profile:
            self.times = [perf_counter()]
            del self.kwargs['rt_profile']
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            if self.profile:
                self.times.append(perf_counter())
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [ ]:
# defining functions used in update

def pred(data):
    
    predictions = np.zeros(np.shape(data))
    predictions[0, :] = nn.predict(data[0, :])
    error = np.mean(np.square(data - predictions), axis=1)
    return error

##########################################################
# change score() to for different action when anomalous! #
##########################################################

def score(score_buffer):

    y_pred = np.mean(score_buffer)
    # just print score for now
    print(y_pred, end='\r')
    
'''  
    if y_pred < 0.510914:
        print("normal,    anomaly score: %f" %y_pred, end="\r")
    else:
        print("anomalous, anomaly score: %f" %y_pred, end="\r")
'''

In [ ]:
# defining functions that will be threaded

def capture(buffer, i):
    '''
    capture function 
    
        captures 1 frame and store in rolling buffer every 0.0000625s (16000 Hz)
    
    args:
    buffer = rolling buffer containing last 1024 frames
    '''
    overlay.audio.record(i/48000)
    buffer[:-i] = buffer[i:]
    # codec records data in 24 bits but then is padded to 32 bits with zeros
    # so we have to convert it to signed 24 bits int then to float to match nn
    for j in range(i):
        if overlay.audio.buffer[-j] > 2**23:
            buffer[-j] = (overlay.audio.buffer[-j] - 2**24) /2**24
        else:
            buffer[-j] = overlay.audio.buffer[-j] /2**24

    
def update(buffer, spectrogram_buffer, score_buffer, init_spectrogram, init_score):
    '''
    continuous processing function
    
        processes data in buffer and updates spectrogram every 0.032s (31.25 Hz)
        if spectrogram is filled, pass spectrogram through nn and print anomaly score

    args:
    buffer = rolling buffer containing last 1024 frames
    spectrogram = rolling mel spectrogram with last 2 time bins
    score_buffer = rolling buffer containing last 196 time slice mse
    init_spectrogram = number of time slices filled in mel spectrogram
    init_score = number of time slices filled in score_buffer
    
    '''
    # processes data in buffer
    temp_buffer = buffer
    temp = com.prep(temp_buffer)
    
    # updates spectrogram
    spectrogram_buffer[:, :-1] = spectrogram_buffer[:, 1:]
    spectrogram_buffer[:, -1:] = temp
    
    # only update score buffer if spectrogram is full
    if init_spectrogram < 2:
        init_spectrogram[0] = init_spectrogram[0] + 1
        return None
    
    temp_spectrogram = spectrogram_buffer
    data = com.transform(temp_spectrogram)
    score_buffer[:-1] = score_buffer[1:]
    score_buffer[-1:] = pred(data)
    
    # only calculate anomaly score if score_buffer is full
    if init_score < 196:
        init_score[0] = init_score[0] + 1
        return None
    
    temp_score = score_buffer
    score(temp_score)

In [ ]:
# start initializing sequence
print('initializing...')
init_spectrogram = np.zeros(1)
init_score = np.zeros(1)
buffer = np.zeros(1024)
spectrogram_buffer = np.zeros([128, 2])
score_buffer = np.zeros(196)

#filling buffer
capture(buffer, 1024)

# create timer object that calls on capture every 0.0000625s (16000 Hz)
rt0 = RepeatedTimer(0.0000625, capture, buffer, 1)
# create timer object that calls on update every 0.032s (31.25 Hz)
rt1 = RepeatedTimer(0.032, update, buffer, spectrogram_buffer, score_buffer, init_spectrogram, init_score)

try:
    #############################################
    # change for sleep()for  different runtime! #
    #############################################
    sleep(50)
finally:
    rt0.stop() 
    rt1.stop()
    return results